# 01 — Create a Normalized SQLite Database (3NF)

This notebook:
- loads the raw `airline.csv`
- cleans column names
- creates a **normalized (3NF)** SQLite database (`data/airline.db`)
- verifies row counts and a sample JOIN query


In [6]:
# Standard libraries
from pathlib import Path
import re
import sqlite3

import pandas as pd

# Reproducibility
SEED = 42

# Paths (works whether you run from project root or /notebooks)
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_DIR = PROJECT_ROOT / "data"
CSV_PATH = DATA_DIR / "airline.csv"
DB_PATH = DATA_DIR / "airline.db"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CSV_PATH:", CSV_PATH, "exists=", CSV_PATH.exists())
print("DB_PATH:", DB_PATH)


PROJECT_ROOT: c:\Users\nepal\OneDrive\Desktop\airline_satisfaction_appp
CSV_PATH: c:\Users\nepal\OneDrive\Desktop\airline_satisfaction_appp\data\airline.csv exists= True
DB_PATH: c:\Users\nepal\OneDrive\Desktop\airline_satisfaction_appp\data\airline.db


## Clean column names
We convert the original columns (with spaces) to snake_case-like names used throughout the project.


In [7]:
def _norm_col(c: str) -> str:
    c = c.strip().lower()
    c = c.replace("departure/arrival", "departure_arrival")
    c = re.sub(r"[^\w]+", "_", c)
    c = re.sub(r"_+", "_", c).strip("_")
    return c

def load_and_clean_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"CSV not found: {path}")

    df = pd.read_csv(path)
    df.columns = [_norm_col(c) for c in df.columns]

    # Common name fixes from the public dataset
    rename_map = {
        "class": "travel_class",
        "departure_delay_in_minutes": "departure_delay_minutes",
        "arrival_delay_in_minutes": "arrival_delay_minutes",
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Target
    if "satisfaction" not in df.columns:
        raise KeyError("Expected a 'satisfaction' column in airline.csv")

    # Normalize target labels
    df["satisfaction"] = df["satisfaction"].astype(str).str.strip().str.lower()
    df["satisfaction_binary"] = (df["satisfaction"] == "satisfied").astype(int)

    # Normalize a few categorical labels (optional but helps consistency)
    for c in ["gender", "customer_type", "type_of_travel", "travel_class"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

    # Basic missing handling
    for c in ["arrival_delay_minutes", "departure_delay_minutes"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # Ensure ints for rating columns
    rating_cols = [
        "inflight_wifi_service",
        "departure_arrival_time_convenient",
        "ease_of_online_booking",
        "gate_location",
        "food_and_drink",
        "online_boarding",
        "seat_comfort",
        "inflight_entertainment",
        "on_board_service",
        "leg_room_service",
        "baggage_handling",
        "checkin_service",
        "inflight_service",
        "cleanliness",
    ]
    for c in rating_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # Age and flight distance
    df["age"] = pd.to_numeric(df["age"], errors="coerce").fillna(df["age"].median()).astype(int)
    df["flight_distance"] = pd.to_numeric(df["flight_distance"], errors="coerce").fillna(df["flight_distance"].median()).astype(int)

    return df

df = load_and_clean_csv(CSV_PATH)
display(df.head())


,unnamed_0,id,gender,customer_type,age,type_of_travel,travel_class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,...,on_board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_minutes,arrival_delay_minutes,satisfaction,satisfaction_binary
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,4,3,4,4,5,5,25,18,neutral or dissatisfied,0
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,5,3,1,4,1,1,6,neutral or dissatisfied,0
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,4,3,4,4,4,5,0,0,satisfied,1
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,5,3,1,4,2,11,9,neutral or dissatisfied,0
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,4,4,3,3,3,0,0,satisfied,1


## 3NF schema design
We normalize into separate tables:

- **passenger**: demographic fields
- **trip**: trip-level fields
- **service_rating**: service scores (1–5)
- **delay**: departure/arrival delay
- **satisfaction**: label table

Each table uses `trip_id` as the primary key. (In this dataset, each row is one trip/passenger record.)


In [8]:
# -----------------------------------------------------------------------------
# Normalized schema (SQLite)
# Each trip is one record in the original dataset.
# We split it into normalized tables with keys + foreign keys.
# -----------------------------------------------------------------------------

PASSENGER_COLS = ["gender", "customer_type", "age"]
TRIP_COLS = ["type_of_travel", "travel_class", "flight_distance"]
RATING_COLS = [
    "inflight_wifi_service",
    "departure_arrival_time_convenient",
    "ease_of_online_booking",
    "gate_location",
    "food_and_drink",
    "online_boarding",
    "seat_comfort",
    "inflight_entertainment",
    "on_board_service",
    "leg_room_service",
    "baggage_handling",
    "checkin_service",
    "inflight_service",
    "cleanliness",
]
DELAY_COLS = ["departure_delay_minutes", "arrival_delay_minutes"]
TARGET_COLS = ["satisfaction", "satisfaction_binary"]

missing = set(PASSENGER_COLS + TRIP_COLS + RATING_COLS + DELAY_COLS + TARGET_COLS) - set(df.columns)
if missing:
    raise KeyError(f"Missing expected columns in cleaned dataframe: {sorted(missing)}")

# Create passenger dimension (unique passengers by demographics)
passengers = df[PASSENGER_COLS].drop_duplicates().reset_index(drop=True)
passengers["passenger_id"] = passengers.index + 1  # 1-based id

df_merged = df.merge(passengers, on=PASSENGER_COLS, how="left")

# Create trip fact table
trips = df_merged[["passenger_id"] + TRIP_COLS].copy().reset_index(drop=True)
trips["trip_id"] = trips.index + 1

# Attach trip_id back
df_trip = df_merged.copy()
df_trip["trip_id"] = trips["trip_id"].values

service = df_trip[["trip_id"] + RATING_COLS].copy()
delays = df_trip[["trip_id"] + DELAY_COLS].copy()
satisfaction = df_trip[["trip_id"] + TARGET_COLS].copy()

display(passengers.head())
display(trips.head())


,gender,customer_type,age,passenger_id
0,Male,Loyal Customer,13,1
1,Male,disloyal Customer,25,2
2,Female,Loyal Customer,26,3
3,Female,Loyal Customer,25,4
4,Male,Loyal Customer,61,5


,passenger_id,type_of_travel,travel_class,flight_distance,trip_id
0,1,Personal Travel,Eco Plus,460,1
1,2,Business travel,Business,235,2
2,3,Business travel,Business,1142,3
3,4,Business travel,Business,562,4
4,5,Business travel,Business,214,5


## Populate tables

In [9]:
# -----------------------------------------------------------------------------
# Write to SQLite (rebuild DB each run)
# -----------------------------------------------------------------------------
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

if DB_PATH.exists():
    DB_PATH.unlink()
    print("Deleted old DB:", DB_PATH)

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

cur.execute("PRAGMA foreign_keys = ON;")

cur.executescript(
    '''
    CREATE TABLE passenger (
        passenger_id INTEGER PRIMARY KEY,
        gender TEXT NOT NULL,
        customer_type TEXT NOT NULL,
        age INTEGER NOT NULL
    );

    CREATE TABLE trip (
        trip_id INTEGER PRIMARY KEY,
        passenger_id INTEGER NOT NULL,
        type_of_travel TEXT NOT NULL,
        travel_class TEXT NOT NULL,
        flight_distance INTEGER NOT NULL,
        FOREIGN KEY(passenger_id) REFERENCES passenger(passenger_id)
    );

    CREATE TABLE service_rating (
        trip_id INTEGER PRIMARY KEY,
        inflight_wifi_service INTEGER,
        departure_arrival_time_convenient INTEGER,
        ease_of_online_booking INTEGER,
        gate_location INTEGER,
        food_and_drink INTEGER,
        online_boarding INTEGER,
        seat_comfort INTEGER,
        inflight_entertainment INTEGER,
        on_board_service INTEGER,
        leg_room_service INTEGER,
        baggage_handling INTEGER,
        checkin_service INTEGER,
        inflight_service INTEGER,
        cleanliness INTEGER,
        FOREIGN KEY(trip_id) REFERENCES trip(trip_id)
    );

    CREATE TABLE delay (
        trip_id INTEGER PRIMARY KEY,
        departure_delay_minutes INTEGER,
        arrival_delay_minutes INTEGER,
        FOREIGN KEY(trip_id) REFERENCES trip(trip_id)
    );

    CREATE TABLE satisfaction (
        trip_id INTEGER PRIMARY KEY,
        satisfaction TEXT NOT NULL,
        satisfaction_binary INTEGER NOT NULL,
        FOREIGN KEY(trip_id) REFERENCES trip(trip_id)
    );
    '''
)

# Insert with pandas
passengers.to_sql("passenger", conn, if_exists="append", index=False)
trips.to_sql("trip", conn, if_exists="append", index=False)
service.to_sql("service_rating", conn, if_exists="append", index=False)
delays.to_sql("delay", conn, if_exists="append", index=False)
satisfaction.to_sql("satisfaction", conn, if_exists="append", index=False)

conn.commit()
conn.close()

print("✅ Database created:", DB_PATH)
print("Rows:")
print(" passenger:", len(passengers))
print(" trip:", len(trips))


Deleted old DB: c:\Users\nepal\OneDrive\Desktop\airline_satisfaction_appp\data\airline.db
✅ Database created: c:\Users\nepal\OneDrive\Desktop\airline_satisfaction_appp\data\airline.db
Rows:
 passenger: 296
 trip: 103904


## Example JOIN query

In [10]:
# -----------------------------------------------------------------------------
# Quick sanity check: reconstruct the dataset via SQL JOIN (used for training)
# -----------------------------------------------------------------------------
conn = sqlite3.connect(DB_PATH)

query = '''
SELECT
    p.gender,
    p.customer_type,
    p.age,
    t.type_of_travel,
    t.travel_class,
    t.flight_distance,
    s.inflight_wifi_service,
    s.departure_arrival_time_convenient,
    s.ease_of_online_booking,
    s.gate_location,
    s.food_and_drink,
    s.online_boarding,
    s.seat_comfort,
    s.inflight_entertainment,
    s.on_board_service,
    s.leg_room_service,
    s.baggage_handling,
    s.checkin_service,
    s.inflight_service,
    s.cleanliness,
    d.departure_delay_minutes,
    d.arrival_delay_minutes,
    sat.satisfaction_binary
FROM trip t
JOIN passenger p ON t.passenger_id = p.passenger_id
JOIN service_rating s ON s.trip_id = t.trip_id
JOIN delay d ON d.trip_id = t.trip_id
JOIN satisfaction sat ON sat.trip_id = t.trip_id
;
'''
df_joined = pd.read_sql_query(query, conn)
conn.close()

display(df_joined.head())
display(df_joined["satisfaction_binary"].value_counts(normalize=True))


,gender,customer_type,age,type_of_travel,travel_class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,...,inflight_entertainment,on_board_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_minutes,arrival_delay_minutes,satisfaction_binary
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18,0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6,0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0,1
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9,0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0,1


satisfaction_binary
0    0.566667
1    0.433333
Name: proportion, dtype: float64